# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [3]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7f8099669ac0>)

In [1]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [2]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [3]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
1575,2.8,1249.250000,-200,15.815633,1174.000000,193.0,711.500000,117.0,1773.50,1486.000000,20.275000,37.450000,0.880692
4525,-200.0,1297.666667,-200,16.054468,1181.333333,-200.0,643.666667,-200.0,1962.00,1255.666667,20.866666,73.300001,1.786970
4267,1.8,1061.500000,-200,9.764202,968.750000,-200.0,728.750000,-200.0,1459.25,782.250000,37.050000,21.175000,1.310703
2991,3.6,1136.500000,-200,16.652685,1199.500000,321.0,647.750000,135.0,1820.75,1489.000000,21.700000,42.350000,1.085900
3273,0.4,851.250000,-200,2.066335,589.000000,-200.0,1119.000000,-200.0,1431.50,500.500000,24.950000,53.125001,1.654106


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [4]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [5]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [6]:
np.random.seed(64224)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [7]:
#Define a function to get the mean valure of each column
def getMean(dfx):
    r ,c = np.shape(dfx)
    mean = np.empty(c)
    for i in range(c):
        mean[i] = ((dfx.iloc[:, i]).sum() - (dfx.iloc[:, i]==-200).sum() * (-200)) / (r - (dfx.iloc[:, i]==-200).sum())
        #mean[i] = np.mean(dfx.iloc[:, i])
    return mean

In [8]:
#Define a function to replace the missing value which is -200 by the mean value
def replace(dfx, o, t):
    r ,c = np.shape(dfx)
    for i in range(c):
        for j in dfx.index:
            if(dfx.iloc[:, i][j] == o):
                dfx.iloc[:, i][j] = t[i]

In [9]:
#Define a function to get the STD which is similar to the getmean function
def getStd(dfx):
    r, c = np.shape(dfx)
    sd = np.empty(c)
    for i in range(c):
        sd[i] = np.std(dfx.iloc[:, i])
    return sd

In [10]:
#Define a function to apply the standardise function
def standardise(dfx, std, smean):
    sdf = dfx
    r ,c = np.shape(dfx)
    for i in range(c):
        for j in dfx.index:
            sdf.iloc[:, i][j] = (dfx.iloc[:, i][j] - smean[i])/std[i]
    return sdf

In [11]:
#Standardise the training dataset by using function above
trainStd = getStd(data_training_unproc)
trainMean = getMean(data_training_unproc)

replace(data_training_unproc, -200, trainMean)
standardise(data_training_unproc, trainStd, trainMean)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8354,2.830376,1.087505,0.215968,0.966002,1.492488,-1.103972,1.947427,0.389534,0.816552,-0.269986,0.575070,-0.006350
7481,-0.799790,-0.767334,-0.178494,-0.917445,-0.508817,0.779213,-0.225322,-1.248101,-0.960526,-0.155519,-0.276386,-0.013352
6482,-0.457322,-0.415630,-0.136924,-0.615666,-0.086860,0.250871,-0.300043,-0.539709,-0.515200,-0.115511,0.427296,0.001230
9250,-1.142259,-0.667368,-0.201582,-1.125053,-0.786848,0.437026,-0.718015,-0.647010,-0.725978,-0.106065,0.442777,0.002233
2418,-0.320334,-0.033765,-0.012072,0.061852,-0.778733,-0.014330,-0.370094,0.558645,-0.633179,0.372364,-0.422283,0.010744
...,...,...,...,...,...,...,...,...,...,...,...,...
1748,-0.525815,0.119953,-0.047491,-0.109133,-0.765921,0.148910,-0.463496,0.331196,-0.478222,0.237892,-0.400704,0.001691
2670,0.227615,0.249836,0.063167,0.390980,-0.620713,-0.509973,-0.183292,1.059381,0.423524,0.215666,0.036049,0.020694
3867,-0.525815,-0.257290,-0.056235,-0.153365,-0.800087,-0.262795,-0.708674,0.516107,-0.483505,0.271788,-0.083577,0.021242
8729,0.775565,0.568705,0.084401,0.477305,1.851237,-0.846753,1.391689,-0.383967,1.571968,-0.159409,-0.191944,-0.012076


## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [15]:
#Set yTrain, xTrain
yTrain = np.array(data_training_unproc.iloc[:, 0].values).reshape(-1,1)
xTrain = np.mat(data_training_unproc.iloc[:,1:].values).A
xTrain = np.hstack((np.ones_like(yTrain), xTrain))

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [16]:
#Standardise the validation set
replace(data_val_unproc, -200, trainMean)
standardise(data_val_unproc, trainStd, trainMean)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
2135,0.775565,-0.090194,0.197802,0.902507,-0.129569,-0.108309,0.096912,0.966665,0.184220,0.351804,-0.548008,0.000642
7453,-0.594309,-0.409063,-0.139849,-0.634928,-0.465255,0.241601,-0.386440,-0.919428,-0.418000,-0.314995,0.594304,-0.008445
6817,1.597489,0.526383,0.098051,0.531525,1.215739,-0.594168,0.633970,-0.168324,0.479520,-0.276654,0.637463,-0.005981
137,1.323515,1.337054,0.222576,0.988832,0.241993,-0.288286,0.540569,1.098447,1.431979,0.084529,-0.296089,-0.003514
9282,-0.731297,-0.549890,-0.145904,-0.675594,-0.364464,0.091750,-0.009332,-1.018394,-1.126929,0.033963,-0.368334,-0.008264
...,...,...,...,...,...,...,...,...,...,...,...,...
2190,-0.799790,-0.511217,-0.064804,-0.197598,-0.894045,0.410764,-0.638623,0.336405,-0.822121,0.143985,-0.108910,0.007331
2382,-0.251841,-0.291585,0.004770,0.139140,-0.586546,-0.101358,-0.124916,0.635909,0.142488,0.141207,-0.034319,0.009967
6695,3.583807,0.000000,0.000000,0.000000,3.850835,0.000000,1.649710,0.000000,0.000000,0.000000,0.000000,0.000000
376,0.638578,0.557030,0.101865,0.546507,-0.108215,-0.159290,0.038537,0.628617,0.746821,-0.062167,-0.093429,-0.005991


In [17]:
#Set several values for each parameters
mini_batch_size_list = np.arange(5,10)
gridSize = 5
learning_rate = np.logspace(-10, 0, 5)
regular_p = np.logspace(-10, 0, 5)
ini_w = np.zeros(shape=(12, 1))

cp_p = {}
cp_p['S'] = []
cp_p['rate'] = []
cp_p['p'] = []

#Store the values for each parameters in dictionary
def setParameter(dateset):
    yTrain = np.array(dateset.iloc[:, 0].values).reshape(-1,1)
    xTrain = np.mat(dateset.iloc[:,1:].values).A
    xTrain = np.hstack((np.ones_like(yTrain), xTrain))
    r ,c = np.shape(dateset)
    
    #S is the interval and the zise is the batch size
    for i in range(len(mini_batch_size_list)):
        S = np.ceil(np.linspace(0, r, mini_batch_size_list[i] + 1))
        random_index = np.random.randint(len(S)-1)
        cp_p['S'] += [S[random_index : random_index + 2].tolist()]
            
    for i in range(len(learning_rate)):
        cp_p['rate'] += [learning_rate[i]]
        cp_p['p'] += [regular_p[i]]
        

In [18]:
setParameter(data_training_unproc) #Prepare the parameters

In [19]:
#return "2𝜂𝑁∑𝑛=1𝑁(𝑦𝑛−𝐱⊤𝑛𝐰old)𝐱𝑛−𝜂𝜆𝐰old" for function start_train
def gradient(x, y, old_w, p, s):
    gradsum = np.zeros(shape=(12,1))
    n1 = int(s[0])
    n2 = int(s[1])
    for i in range(n1 , n2):
        gradsum += (y[i] - x[i, :].dot(old_w)) * x[i, :].reshape(-1,1)
    return ((2 * gradsum) / (n2 - n1)) - (p * old_w) 

In [20]:
#trainning fauntion to update w
def start_train(x, y, w, rate, p, s):
    old_w = w
    for i in range(200):   
        new_w = old_w + rate * gradient(x, y, old_w, p, s) #update new w
        old_w = new_w
    return new_w

In [21]:
re_w_list = [] #the list of w after every train
s_list = [] #the list of "S" parameter
for i in range(gridSize):
    re_w = []
    s_mid = []
    for j in range(gridSize):
        re_w.append(start_train(xTrain, yTrain, ini_w, cp_p['rate'][i], cp_p['p'][j], cp_p['S'][i]))
        s_mid.append(cp_p['S'][i])
    re_w_list.append(re_w)
    s_list.append(s_mid)

#[rate][p]

In [22]:
#Define a function to calculate the RMSE
def getRMSE(x, y, w, s):
    rmse = 0
    n1 = int(s[0])
    n2 = int(s[1])
    n = n2 - n1
    for i in range(n1, n2):
        rmse += (y[i] - x[i, :].dot(w)) ** 2
    return (rmse / n) ** 0.5

In [23]:
#return the best value which is the min RMSE and the index of the parameter in the list stored before
def getBest(x, y, rw, s):
    index = [0, 0]
    minRMSE = float('inf')
    for i in range(gridSize):
        for j in range(gridSize):
            thisRMSE = getRMSE(x, y, rw[i][j], s[i][j])
            if (thisRMSE < minRMSE):
                minRMSE = thisRMSE
                index = [i, j]
    return minRMSE, index


In [24]:
train_re = {} #train restult
result = getBest(xTrain, yTrain, re_w_list, s_list)
re_index = result[1] #get the index of the best parameters in the list
#store each best parameter after trainning
train_re['S'] = s_list[re_index[0]][re_index[1]]
train_re['rate'] = cp_p['rate'][re_index[0]]
train_re['p'] = cp_p['p'][re_index[1]]
train_re['w'] = re_w_list[re_index[0]][re_index[1]]

In [25]:
#show the trainning result with best parameters
train_re

{'S': [0.0, 672.0],
 'rate': 0.0031622776601683794,
 'p': 1e-10,
 'w': array([[ 0.00125235],
        [ 0.19507739],
        [ 0.06333953],
        [ 0.24911176],
        [ 0.30175822],
        [-0.11724876],
        [ 0.15228673],
        [ 0.17745483],
        [ 0.20807243],
        [-0.00537053],
        [-0.00420931],
        [-0.00052789]])}

## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [26]:
#because I changed the training set and the validation set before so I have to store the original data again to put train and val data together and standardise them

np.random.seed(64224)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

In [27]:
#same operation as training set
train_val = pd.concat([data_training_unproc, data_val_unproc])

In [28]:
newStd = getStd(train_val)
newMean = getMean(train_val)

In [29]:
replace(train_val, -200, newMean)
standardise(train_val, newStd, newMean)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8354,2.845879,1.089576,0.216287,0.972891,1.496695,-1.105438,1.949606,0.394431,0.824262,-0.269257,0.572650,-0.006336
7481,-0.797102,-0.762978,-0.176128,-0.912481,-0.504973,0.771374,-0.220145,-1.244509,-0.951882,-0.155467,-0.274122,-0.013297
6482,-0.453424,-0.411707,-0.134775,-0.610393,-0.082940,0.244820,-0.294763,-0.535553,-0.506790,-0.115696,0.425689,0.001199
9250,-1.140779,-0.663136,-0.199098,-1.120300,-0.783054,0.430346,-0.712157,-0.642939,-0.717456,-0.106305,0.441085,0.002196
2418,-0.315953,-0.030313,-0.010570,0.067817,-0.774938,-0.019484,-0.364718,0.563677,-0.624707,0.369293,-0.419216,0.010657
...,...,...,...,...,...,...,...,...,...,...,...,...
2190,-0.797102,-0.507177,-0.063028,-0.191897,-0.890271,0.404172,-0.632876,0.341259,-0.813550,0.142265,-0.107567,0.007264
2382,-0.247218,-0.287815,0.006185,0.145184,-0.582716,-0.106217,-0.119877,0.641002,0.150552,0.139503,-0.033387,0.009884
6695,3.601969,0.000000,0.000000,0.000000,3.855468,0.000000,1.652300,0.000000,0.000000,0.000000,0.000000,0.000000
376,0.646343,0.559754,0.102776,0.552967,-0.104298,-0.163953,0.043350,0.633704,0.754568,-0.062667,-0.092171,-0.005979


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [30]:
#same operation as training set
testStd = getStd(data_test_unproc)
testMean = getMean(data_test_unproc)

In [31]:
replace(data_test_unproc, -200, testMean)
standardise(data_test_unproc, testStd, testMean)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
4273,-0.217310,-0.011326,-0.024326,0.004339,0.000000,-0.319475,0.000000,0.153106,-0.308570,0.334516,-0.400652,0.009521
349,-0.702449,-0.197287,-0.152574,-0.719722,-0.721764,1.266959,-0.363160,-0.146979,-0.923493,-0.232834,0.577601,-0.003933
6595,-0.217310,-0.555600,-0.048890,-0.115971,0.249467,0.035228,0.023537,-0.936760,-0.371895,-0.001882,-0.534686,-0.014468
7193,0.822273,0.000000,0.000000,0.000000,1.569816,0.000000,1.012031,0.000000,0.000000,0.000000,0.000000,0.000000
4906,-1.048977,-0.603224,-0.169506,-0.842966,-0.734888,0.391427,-0.701520,-0.267544,-0.436887,0.035368,0.350033,0.013280
...,...,...,...,...,...,...,...,...,...,...,...,...
6390,0.059912,-0.083896,-0.048598,-0.114504,0.293216,2.364741,-0.121475,-0.259577,-0.138035,-0.150883,0.617137,0.002511
3758,-0.633143,-0.407437,-0.079449,-0.275162,-0.721764,-0.111442,-0.532340,0.369805,-0.290794,0.155715,0.080037,0.016603
8017,0.129218,-0.006791,-0.045815,-0.100566,0.200905,-0.389816,1.114748,-0.907548,0.165260,-0.238564,-0.219851,-0.015465
6295,0.129218,0.120962,-0.061127,-0.178327,0.175093,-0.297773,-0.314823,-0.408291,0.106934,-0.170368,0.548673,-0.000167


In [32]:
setParameter(train_val)

In [33]:
val_w = start_train(xTrain, yTrain, ini_w, train_re['rate'], train_re['p'], train_re['S']) # get the w using the best parameter got before

In [34]:
val_w #show w

array([[ 0.00125235],
       [ 0.19507739],
       [ 0.06333953],
       [ 0.24911176],
       [ 0.30175822],
       [-0.11724876],
       [ 0.15228673],
       [ 0.17745483],
       [ 0.20807243],
       [-0.00537053],
       [-0.00420931],
       [-0.00052789]])

In [35]:
#set X, y for test set
yTest = np.array(data_test_unproc.iloc[:, 0].values).reshape(-1,1)
xTest = np.mat(data_test_unproc.iloc[:,1:].values).A
xTest = np.hstack((np.ones_like(yTest), xTest))

In [36]:
#get RMSE for test set
test_RMSE = getRMSE(xTest, yTest, val_w, [0, len(yTest)])

In [37]:
test_RMSE

array([0.40818138])

# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [12]:
#again...
np.random.seed(64224)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#usign pipe to build a list of methods to standardise the data
imp_mean = SimpleImputer(missing_values = -200, strategy = 'mean')
pipe = Pipeline([('imp', imp_mean), ('scaler', StandardScaler())])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [14]:
#apply pipe operation
pipe.fit(data_training_unproc)
train_data = pipe.transform(data_training_unproc)

In [15]:
pipe.transform(data_val_unproc)

array([[ 7.75564978e-01, -1.43376802e-01,  1.16215616e+00, ...,
         1.82972209e+00, -1.72577043e+00,  6.71041661e-02],
       [-5.94309114e-01, -6.50267678e-01, -8.21659988e-01, ...,
        -1.63828151e+00,  1.87156432e+00, -8.82264057e-01],
       [ 1.59748943e+00,  8.36766881e-01,  5.76083155e-01, ...,
        -1.43887129e+00,  2.00748000e+00, -6.24768350e-01],
       ...,
       [ 3.58380687e+00,  1.05495324e-15,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  5.68901690e-16],
       [ 6.38577569e-01,  8.85484082e-01,  5.98491837e-01, ...,
        -3.23330135e-01, -2.94223248e-01, -6.25811828e-01],
       [ 1.59121637e-01, -7.61006680e-02, -3.90369832e-01, ...,
        -1.47066134e+00,  1.83019873e+00, -7.27171189e-01]])

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import cross_val_score

In [17]:
#set X,y for training
X = train_data[:, 1:]
Y = train_data[:, 0]

In [18]:
#use PredefinedSplit to slice the data for train and validation
d = PredefinedSplit(np.random.randint(-1, 2, len(Y)))
for train_index, test_index in d.split():
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

In [21]:
#find best n_estimators
param_n_estimators = {'n_estimators' : np.linspace(1, 101, 11).astype('int')}
gs = GridSearchCV(estimator = RandomForestClassifier(random_state = 64224), param_grid = param_n_estimators, cv = 10)
gs.fit(X_train, Y_train.astype('int'))

print(gs.best_score_)
print(gs.best_params_)

/Users/chaselth/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.8745535437944909
{'n_estimators': 21}


In [22]:
#find best max_features
param_max_features = {'max_features' : np.linspace(1, 10, 10).astype('int')}
gs = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 51, random_state = 64224),
                  param_grid = param_max_features, cv = 10)
gs.fit(X_train, Y_train.astype('int'))
print(gs.best_score_)
print(gs.best_params_)

/Users/chaselth/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.8765026307644692
{'max_features': 2}


In [23]:
#find best max_samples
param_max_samples = {'max_samples' : np.linspace(11, 20, 10).astype('int')}
gs = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 51,
                                                     random_state = 64224,
                                                     max_features = 2),
                  param_grid = param_max_samples, cv = 10)
gs.fit(X_train, Y_train.astype('int'))
print(gs.best_score_)
print(gs.best_params_)

/Users/chaselth/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.8119653358093469
{'max_samples': 19}


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [24]:
train_val = pd.concat([data_training_unproc, data_val_unproc])
train_val = pipe.fit_transform(train_val)
test_data = pipe.fit_transform(data_test_unproc)

In [55]:
X = train_val[:, 1:]
Y = train_val[:, 0]
x_test = test_data[:, 1:]
y_test = test_data[:, 0]

In [26]:
#set the random forest
test_model = RandomForestClassifier(n_estimators = 21,
                                    random_state = 64224,
                                    max_features = 2,
                                    max_samples = 19)

In [66]:
test_model.fit(x_test, y_test.astype('int'))
score = test_model.score(x_test, y_test.astype('int'))
score

0.7984361424847958

In [67]:
#calculate the RMSE
rmse = 0
for i in range(len(y_test)):
    rmse += (y_test[i] - test_pred[i]) ** 2
rmse = (rmse / len(y_test)) ** 0.5
print(rmse)

0.8010009571985901
